In [32]:
import pandas as pd

# URL do Wikipedije
url = "https://en.wikipedia.org/wiki/List_of_Academy_Award%E2%80%93winning_films"

# Branje tabel iz URL-ja
tables = pd.read_html(url)

# Preverjanje števila tabel
print(f"Najdene tabele: {len(tables)}")

# Izbor želene tabele (privzeto je prva tabela prava)
awards_table = tables[0]

# Prikaz prvih vrstic za pregled
print(awards_table.head())

# Preimenovanje stolpcev za boljšo berljivost
awards_table.columns = [
    "naslov",
    "leto",
    "awards_won",
    "awards_nominated"
]

# Odstranitev morebitnih praznih vrstic
awards_table = awards_table.dropna()

# Pretvorba leta v prvo število pred znakom '/'
awards_table["leto"] = awards_table["leto"].str.split('/').str[0]

# Čiščenje podatkov v stolpcih "awards_won" in "awards_nominated"
awards_table["awards_won"] = awards_table["awards_won"].str.extract(r"(\d+)").astype(int)
awards_table["awards_nominated"] = awards_table["awards_nominated"].str.extract(r"(\d+)").astype(int)

# Priprava tabele z razdeljenimi vrsticami
expanded_rows = []
for _, row in awards_table.iterrows():
    won = row["awards_won"]
    nominated = row["awards_nominated"]
    not_won = nominated - won
    # Dodaj vrstico za FALSE (kolikokrat ni zmagal)
    expanded_rows.append([row["naslov"], row["leto"], "FALSE", not_won])
    # Dodaj vrstico za TRUE (kolikokrat je zmagal)
    expanded_rows.append([row["naslov"], row["leto"], "TRUE", won])

# Ustvarjanje nove tabele
final_table = pd.DataFrame(expanded_rows, columns=["naslov", "leto", "oskar", "stevilo"])

# Odstranitev filmov, kjer so vse nominacije in zmage enake 0
final_table = final_table.groupby("naslov").filter(lambda x: not (x["stevilo"].sum() == 0))

# Izvoz v CSV
destination_csv = "academy_awards_filtered.csv"
final_table.to_csv(destination_csv, index=False)

print(f"Tabela je bila uspešno shranjena kot {destination_csv}")

Najdene tabele: 2
                   Film  Year Awards Nominations
0           Oppenheimer  2023      7          13
1           Poor Things  2023      4          11
2  The Zone of Interest  2023      2           5
3      American Fiction  2023      1           5
4     Anatomy of a Fall  2023      1           5
Tabela je bila uspešno shranjena kot academy_awards_filtered.csv
